<a href="https://colab.research.google.com/github/diogofn1/Healthym-RAG/blob/main/Knowledge_base_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Important
This notebook was made to be run in google colab. This was done in order to allow users to make use of it regardless of having a good computer at home. However, with a small change in loading Hugging Face token, it can be run in a personal computer.
If run as a free Google Colab user, I recommend connection to a machine called 'GPUs: T4'. You can set that at the top right of your screen.

# Synthetic text data generator
The goal of this notebook is to use a synthetic text data generator to make a knowledge base about a company caled Healthym, which is a fictional company in the healthy foods market. The data produced should cover information company, recipes and suppliers.

For a more detailed approach on how the data generator works or to use it in your own projects, please consult: https://github.com/diogofn1/Synthetic-Text-Data-Generator

In [ ]:
# Install requirements

!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.2 MB/s eta 0:00:00


In [ ]:
# Relevant imports

from google.colab import userdata, files
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gc
from IPython import display

In [ ]:
# Login to HuggingFace. HF_Token must be set in Google Colab Secrets
# If using on a personal machine, change this cell to load HF_TOKEN from a .env file
# One could also hardcode the token, but it is not recommended

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Quantization configuration
# Quantization allows less use of memory to load the model at the cost of some accuracy
# Without this, the notebook is likely to crash for lack of memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
# Load desired model

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct",
                                             device_map="auto",
                                             quantization_config=quant_config)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

## Generating new data
### 1. Company description
The description of the company was already created using this generator at a prior date. It will be use as a base to take decisions about products, recipes and market campaing to be generated in this notebook.

In [ ]:
# Load in company description
company_summary = open('./knowledge_base/company/company_summary.md', 'r').read()
display.Markdown(company_summary)

==============================================

Healthym: Nourishing Communities with Healthy Foods
=====================================================

### Summary

Healthym is a pioneering company dedicated to providing high-quality, healthy foods to local communities. We believe that everyone deserves access to nutritious food, regardless of their location or socioeconomic status. Our mission is to make healthy eating a reality for all, while promoting sustainability and supporting local economies.

### Services

Healthym offers a wide range of healthy food products, including:

* Fresh produce: seasonal fruits and vegetables sourced from local farmers
* Whole grains: artisanal bread, pasta, and rice from small-scale producers
* Plant-based protein: organic tofu, tempeh, and seitan from local suppliers
* Specialty foods: artisanal cheeses, fermented foods, and international cuisine
* Meal kits: pre-portioned ingredients and recipes for easy meal prep

Our products are carefully curated to meet the diverse needs of our customers, from busy professionals to families and individuals with dietary restrictions.

### Background History

Healthym was founded in 2015 by a group of passionate entrepreneurs who recognized the need for healthier food options in local communities. Our initial store opened in a small town in the Midwest, where we quickly gained a loyal customer base. As our popularity grew, we expanded to new locations, refining our business model and product offerings along the way.

Today, Healthym operates over 50 stores across the United States, with a strong presence in urban and rural areas. Our commitment to quality, sustainability, and community engagement has earned us a reputation as a trusted leader in the healthy food industry.

### Company Values and Mission

At Healthym, we live by the following values:

* **Quality**: We source only the best ingredients from local suppliers to ensure our products meet the highest standards.
* **Sustainability**: We strive to minimize our environmental footprint through eco-friendly packaging, reduced waste, and energy-efficient operations.
* **Community**: We believe in giving back to the communities we serve, through partnerships with local organizations and initiatives that promote food security and education.

Our mission is to:

* Provide access to healthy, nutritious food for all members of our communities
* Foster a culture of sustainability and environmental responsibility
* Support local economies and small-scale producers
* Educate and empower our customers to make informed food choices

At Healthym, we're dedicated to making a positive impact on the health and well-being of our customers, while promoting a more sustainable and equitable food system. Join us in our mission to nourish communities and create a healthier, happier world.

In [ ]:
# Define helper function to generate new data

def generate_document(prompt, max_new_tokens=2000):

  """
  Generate documents based on the prompt given.
  Prompt (str): Instructions for the model.
  max_new_tokens (int): Maximum number of new tokens to generate.
  """

  prompt_updated = (
    "You are a synthetic data generator.\n\n"
    f"{prompt}\n"
    "Output your response in markdown format.\n"
  )
  inputs = tokenizer(prompt_updated, return_tensors="pt").to("cuda")
  input_length = inputs['input_ids'].shape[1]  # Number of tokens in the prompt
  outputs = model.generate(inputs['input_ids'], max_new_tokens=2000)
  generated_tokens = outputs[0][input_length:]
  generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

  return generated_text

### 2. Products

In [ ]:
# Generate a list of products

input_prompt = "Generate a list of 15 products commercialized by Healthym, a company that sells healthy foods at local stores.\n"
input_prompt += "Use the company description as context.\n"
input_prompt += f"Company description:\n\n {company_summary}\n\n"
input_prompt += "Here are some product examples: seasonal fruits, artisanal bread, artisanal cheeses, gluten-free pasta."

inputs = tokenizer(input_prompt, return_tensors="pt").to("cuda")
input_length = inputs['input_ids'].shape[1]  # Number of tokens in the prompt
outputs = model.generate(inputs['input_ids'], max_new_tokens=500)
generated_tokens = outputs[0][input_length:]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
# Inspect generated list

display.Markdown(generated_text)

 There are many more products that Healthym sells. Here is a list of 15 products commercialized by Healthym:

1.  **Fresh Pineapple**: A sweet and juicy seasonal fruit sourced from local farmers.
2.  **Whole Wheat Bread**: Artisanal bread made from high-quality whole wheat flour and baked in small batches.
3.  **Greek Yogurt**: A thick and creamy yogurt made from non-fat milk and live cultures.
4.  **Quinoa Salad**: A mix of cooked quinoa, roasted vegetables, and a tangy dressing, perfect for a quick and easy meal.
5.  **Roasted Chickpeas**: Crispy and flavorful chickpeas seasoned with herbs and spices, great as a snack or addition to salads.
6.  **Grilled Chicken Breast**: Lean and juicy chicken breast, marinated in a blend of herbs and spices, perfect for sandwiches or salads.
7.  **Brown Rice**: High-quality brown rice, sourced from local suppliers and cooked to perfection.
8.  **Spinach and Feta Wrap**: A flavorful wrap filled with spinach, feta cheese, and a hint of lemon, perfect for a quick and easy meal.
9.  **Cauliflower Rice**: A low-carb and gluten-free alternative to traditional rice, made from cauliflower and perfect for stir-fries and curries.
10. **Apple Cider**: A refreshing and tangy apple cider, made from locally sourced apples and perfect for hot summer days.
11. **Kale Chips**: Crispy and flavorful kale chips, seasoned with herbs and spices, great as a snack or addition to salads.
12. **Turkey Meatballs**: Lean and flavorful turkey meatballs, made with ground turkey and a blend of herbs and spices, perfect for pasta sauces and sandwiches.
13. **Gluten-Free Pasta**: A high-quality gluten-free pasta, made from rice flour and perfect for those with dietary restrictions.
14. **Avocado Toast**: Toasted whole grain bread topped with mashed avocado, cherry tomatoes, and a sprinkle of red pepper flakes, perfect for a healthy and delicious breakfast or snack.
15. **Lentil Soup**: A hearty and comforting lentil soup, made with red lentils and a blend of aromatic spices, perfect for a cold winter's day.

These are just a few examples of the many products that Healthym sells. Their product line is diverse and caters to various dietary needs and preferences. Healthym

This is a great list to work with. In order to refine it, "Fresh Pineapple" will be replaced by Fresh Fruits; and "Whole Wheat Bread" by "Artisanal Bread".

In [ ]:
# Make refinements

product_list = generated_text.replace("Fresh Pineapple**: A sweet and juicy seasonal fruit", "Fresh Fruits**: Sweet and juicy seasonal fruits")
product_list = product_list.replace("Whole Wheat Bread", "Artisanal Bread")


In [ ]:
display.Markdown(product_list)

 There are many more products that Healthym sells. Here is a list of 15 products commercialized by Healthym:

1.  **Fresh Fruits**: Sweet and juicy seasonal fruits sourced from local farmers.
2.  **Artisanal Bread**: Artisanal bread made from high-quality whole wheat flour and baked in small batches.
3.  **Greek Yogurt**: A thick and creamy yogurt made from non-fat milk and live cultures.
4.  **Quinoa Salad**: A mix of cooked quinoa, roasted vegetables, and a tangy dressing, perfect for a quick and easy meal.
5.  **Roasted Chickpeas**: Crispy and flavorful chickpeas seasoned with herbs and spices, great as a snack or addition to salads.
6.  **Grilled Chicken Breast**: Lean and juicy chicken breast, marinated in a blend of herbs and spices, perfect for sandwiches or salads.
7.  **Brown Rice**: High-quality brown rice, sourced from local suppliers and cooked to perfection.
8.  **Spinach and Feta Wrap**: A flavorful wrap filled with spinach, feta cheese, and a hint of lemon, perfect for a quick and easy meal.
9.  **Cauliflower Rice**: A low-carb and gluten-free alternative to traditional rice, made from cauliflower and perfect for stir-fries and curries.
10. **Apple Cider**: A refreshing and tangy apple cider, made from locally sourced apples and perfect for hot summer days.
11. **Kale Chips**: Crispy and flavorful kale chips, seasoned with herbs and spices, great as a snack or addition to salads.
12. **Turkey Meatballs**: Lean and flavorful turkey meatballs, made with ground turkey and a blend of herbs and spices, perfect for pasta sauces and sandwiches.
13. **Gluten-Free Pasta**: A high-quality gluten-free pasta, made from rice flour and perfect for those with dietary restrictions.
14. **Avocado Toast**: Toasted whole grain bread topped with mashed avocado, cherry tomatoes, and a sprinkle of red pepper flakes, perfect for a healthy and delicious breakfast or snack.
15. **Lentil Soup**: A hearty and comforting lentil soup, made with red lentils and a blend of aromatic spices, perfect for a cold winter's day.

These are just a few examples of the many products that Healthym sells. Their product line is diverse and caters to various dietary needs and preferences. Healthym

In [ ]:
# Save product list locally
with open(f'product_sample.md', 'w') as f:
  f.write(product_list)
files.download(f'product_sample.md')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Generate products decriptions based on product list

products = ['seasonal fruits', "artisanal bread", "greek yorgut", "quinoa salad", "roasted chickpeas",
            "grilled chicken breast", "brown rice", "spinach and feta wrap", "cauliflower rice", "apple cider",
            "kale chips", "turkey meatballs", "gluten-free pasta", "avocado toast", "lentil soup"]

for product in products:

  prompt = (
      "Generate a document about products commercialized by Healthym, a company that sells healthy foods at local stores.\n"
      "This document should contain a couple of sections: product summary; a description of what it can be used for;\n"
      "product origin; and a sample recipe in which it can be used, if applicable.\n"
      f"The product is: {product}.\n"
  )

  product_description = generate_document(prompt, max_new_tokens=2000)

  with open(f'{product}.md', 'w') as f:
    f.write(product_description)
  files.download(f'{product}.md')

In [ ]:
# Product file sample
with open("quinoa salad.md", 'r') as f:
  file = f.read()

display.Markdown(file)

# Healthym Product Summary

## Quinoa Salad

### Product Summary

Quinoa salad is a delicious and nutritious food product from Healthym, made with quinoa, mixed vegetables, and a hint of lemon juice. This product is perfect for health-conscious individuals who want to incorporate more plant-based meals into their diet.

### Description of What It Can Be Used For

Quinoa salad can be used as a:

* Main course: Serve it as a filling and satisfying meal on its own or with some whole grain bread.
* Side dish: Add it to your favorite meals, such as grilled chicken or fish, for a nutritious and tasty accompaniment.
* Salad topping: Mix it with your favorite greens, nuts, and fruits to create a fresh and healthy salad.
* Meal prep: Use it as a base for your meal prep bowls, adding your favorite proteins and vegetables.

### Product Origin

Quinoa salad is made from high-quality quinoa, sourced from local farmers in the region. The mixed vegetables are carefully selected and washed to ensure freshness and quality. The product is then prepared and packaged in our state-of-the-art facility, adhering to the highest food safety standards.

### Sample Recipe

**Quinoa Salad with Roasted Vegetables and Lemon Vinaigrette**

Ingredients:

* 1 cup quinoa salad
* 2 cups mixed roasted vegetables (such as broccoli, carrots, and sweet potatoes)
* 2 tablespoons lemon vinaigrette
* 1/4 cup chopped fresh herbs (such as parsley and basil)
* Salt and pepper to taste

Instructions:

1. Preheat your oven to 400°F (200°C).
2. Toss the mixed vegetables with olive oil, salt, and pepper, and roast in the oven for 20-25 minutes or until tender.
3. In a large bowl, combine the quinoa salad, roasted vegetables, and chopped fresh herbs.
4. Drizzle with lemon vinaigrette and toss to combine.
5. Serve warm or at room temperature.

Enjoy your delicious and nutritious quinoa salad!

### 3. Recipes

In [ ]:
# Generate recipes for different occasions based on products

products = ['seasonal fruits', "artisanal bread", "greek yorgut", "quinoa salad", "roasted chickpeas",
            "grilled chicken breast", "brown rice", "spinach and feta wrap", "cauliflower rice", "apple cider",
            "kale chips", "turkey meatballs", "gluten-free pasta", "avocado toast", "lentil soup"]

occasions = ['breakfast', 'lunch', 'diner']

for product in products:
  for occasion in occasions:

    prompt = (
        "Generate a document about a recipe for a specific occasion using a product commercialized by Healthym, a company that sells healthy foods at local stores.\n"
        "This document should contain a couple of sections: recipe title;"
        "recipe summary (includeing ingredients list, how long it takes to make it, and how many people it serves);\n"
        "how to make it step-by-step."
        f"The product is: {product}.\n"
        f"The occasion is: {occasion}.\n"
    )

    product_description = generate_document(prompt, max_new_tokens=2000)

    with open(f'{product}_{occasion}_recipe.md', 'w') as f:
      f.write(product_description)
    files.download(f'{product}_{occasion}_recipe.md')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Recipe file sample
with open("greek yorgut_breakfast_recipe.md", 'r') as f:
  file = f.read()

display.Markdown(file)

### Greek Yogurt Breakfast Bowl Recipe

#### Recipe Summary

*   **Title:** Greek Yogurt Breakfast Bowl
*   **Ingredients:**
    *   1 cup Greek yogurt (Healthym's Greek Yorgut)
    *   1/2 cup mixed berries (fresh or frozen)
    *   1 tablespoon honey
    *   1/2 cup granola
    *   1/4 cup chopped walnuts
*   **Prep Time:** 5 minutes
*   **Cook Time:** 0 minutes
*   **Servings:** 1

#### How to Make It Step-by-Step

1.  In a small bowl, mix together the Greek yogurt and honey until well combined.
2.  Add the mixed berries to the yogurt mixture and gently fold them in.
3.  In a separate bowl, mix together the granola and chopped walnuts.
4.  Spoon the yogurt and berry mixture into a bowl or glass.
5.  Top the yogurt mixture with the granola and walnut mixture.
6.  Serve immediately and enjoy!

This recipe is perfect for a quick and healthy breakfast on-the-go. The Greek yogurt provides a good source of protein, while the mixed berries add natural sweetness and the granola and walnuts provide a crunchy texture. This recipe is also versatile and can be customized to your taste preferences. You can use different types of berries or add other toppings such as banana slices or shredded coconut. Enjoy your delicious and nutritious Greek Yogurt Breakfast Bowl!

### 4. Suppliers

In [ ]:
# Generate information about Healthym supliers. Use company descrition as a base to start

# Generate a list of supliers

input_prompt = "Generate a list of 5 suppliers that work with Healthym, a company that sells healthy foods at local stores.\n"
input_prompt += "Use the company description as context.\n"
input_prompt += f"Company description:\n\n {company_summary}\n\n"
input_prompt += "Give each suplier a name and a short description of what is provides."

inputs = tokenizer(input_prompt, return_tensors="pt").to("cuda")
input_length = inputs['input_ids'].shape[1]  # Number of tokens in the prompt
outputs = model.generate(inputs['input_ids'], max_new_tokens=500)
generated_tokens = outputs[0][input_length:]
suppliers_list = tokenizer.decode(generated_tokens, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
display.Markdown(suppliers_list)

  Here is the list of 5 suppliers:

1. **Green Earth Produce Co.**
	* Description: Green Earth Produce Co. specializes in organic and locally grown produce, offering a wide variety of fruits and vegetables that meet Healthym's high standards for quality and sustainability.
2. **Artisan's Grain Mill**
	* Description: Artisan's Grain Mill is a small-scale producer of artisanal bread, pasta, and rice. They use traditional methods and high-quality ingredients to create unique and delicious products that cater to Healthym's customers.
3. **PureProtein**
	* Description: PureProtein is a local supplier of organic tofu, tempeh, and seitan. They use a proprietary process to ensure the highest quality and nutritional content of their products, which are carefully selected by Healthym's team.
4. **Fermentia Foods**
	* Description: Fermentia Foods is a specialty food supplier that offers a range of artisanal cheeses, fermented foods, and international cuisine. Their products are carefully crafted to meet Healthym's standards for quality, taste, and nutritional value.
5. **Sunny Meadows Meal Kits**
	* Description: Sunny Meadows Meal Kits provides pre-portioned ingredients and recipes for easy meal prep. Their products are designed to cater to Healthym's customers' busy lifestyles, while promoting healthy eating and sustainability.

These suppliers work closely with Healthym to ensure the highest quality and consistency of their products, which are carefully selected to meet the diverse needs of their customers. By partnering with local suppliers, Healthym promotes sustainability, supports local economies, and fosters a culture of community engagement.

In [ ]:
# Save it locally
with open(f'suppliers_list.md', 'w') as f:
  f.write(suppliers_list)
files.download(f'suppliers_list.md')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Generate suppliers information

suppliers = ["Green Earth Produce Co", "Artisan's Grain Mill", "PureProtein", "Fermentia Foods", "Sunny Meadows Meal Kits"]

for supplier in suppliers:


  prompt = (
      "Generate a document about a supplier that works with Healthym, a company that sells healthy foods at local stores.\n"
      "This document should contain a brief description of what the supplier provides and why Healthym works with them;"
      "Use both the 'Company summary' and 'Suppliers summary' as context;\n\n"
      f"Company summary:\n {company_summary}.\n\n"
      f"Suppliers summary: {suppliers_list}.\n\n"
      f"The supplier is {supplier}"
  )

  supplier_description = generate_document(prompt, max_new_tokens=1000)

  with open(f'{supplier}.md', 'w') as f:
    f.write(supplier_description)
  files.download(f'{supplier}.md')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Supplier file sample

with open("Artisan's Grain Mill.md", 'r') as f:
  file = f.read()

display.Markdown(file)

=====================================

### Supplier: Artisan's Grain Mill

**Description**
---------------

Artisan's Grain Mill is a small-scale producer of artisanal bread, pasta, and rice. They use traditional methods and high-quality ingredients to create unique and delicious products that cater to Healthym's customers.

**Why Healthym Works with Artisan's Grain Mill**
----------------------------------------------

Healthym values Artisan's Grain Mill for their commitment to quality, sustainability, and community engagement. Here are some reasons why we partner with them:

*   **High-quality products**: Artisan's Grain Mill produces artisanal bread, pasta, and rice that meet Healthym's high standards for quality and taste.
*   **Sustainable practices**: They use traditional methods and locally sourced ingredients to minimize their environmental footprint.
*   **Community engagement**: Artisan's Grain Mill supports local farmers and artisans, promoting a culture of sustainability and community engagement.
*   **Collaborative approach**: We work closely with Artisan's Grain Mill to ensure the highest quality and consistency of their products, which are carefully selected to meet the diverse needs of our customers.

**Benefits of Partnering with Artisan's Grain Mill**
---------------------------------------------------

By partnering with Artisan's Grain Mill, Healthym benefits from:

*   **Unique and delicious products**: Artisan's Grain Mill's products are carefully crafted to meet our customers' expectations for quality, taste, and nutritional value.
*   **Sustainable and eco-friendly practices**: Their commitment to sustainability aligns with Healthym's values and promotes a more environmentally responsible food system.
*   **Community engagement and support**: Artisan's Grain Mill's community-focused approach resonates with Healthym's mission to nourish communities and create a healthier, happier world.

**Conclusion**
----------

Artisan's Grain Mill is a valued supplier of Healthym, offering high-quality, artisanal bread, pasta, and rice that meet our customers' needs. By partnering with them, we promote sustainability, support local economies, and foster a culture of community engagement. Join us in our mission to nourish communities and create a healthier, happier world. Together, we can make a positive impact on the health and well-being of our customers and the environment.

## Final remarks

This notebook can be use to generate synthetic text data for multiple purposes. The examples dispalyed here showcases its use to generate artificial data for a company called Healthym: one file for a company descrition and multiple files describing the product it offers. The user can adapt the parameters of the function generate_document() to fit their needs and address other contexts.